In [28]:
from treeParse import parseTree
from anytree import Node, RenderTree
import sys
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")
import site
from copy import deepcopy
import re
import string
import zss

def insert_cost(A):
    #print("inserting "+A.my_label)
    cost=3
    return cost
    
def remove_cost(A):
    #print("deleting "+A.my_label)
    cost=3
    return cost

def update_cost(A,B):
    incost=0
    if(A.upostag!=B.upostag):
        incost+=1
    if(A.depRel!=B.depRel):
        incost+=1
    if(A.form==B.form):
        cost=incost
    else:
        cost=remove_cost(A)+insert_cost(B)
    return cost

class Extractor():
    
    def __init__(self, id2, form, lemma, upostag, xpostag, feats,head,deprel,deps,misc):
        self.id = id2
        self.form = form
        self.lemma = lemma
        self.upostag = upostag
        self.xpostag = xpostag
        self.feats = feats
        self.head=head
        self.deprel=deprel
        self.deps=deps
        self.misc=misc
    
    def get_children():
        return node.my_children
    
    def get_nodeID():
        return node.nodeID
    
    def get_depRel():
        return node.depRel
    
    def get_parentID():
        return node.parentID
    
    def get_posTag():
        return node.posTag
    
    def get_lemma():
        return node.my_lemma
extract_holder=[]

c=0

def lookup(postag):
    #print(postag)
    if(postag=="NN" or postag=="NNS"):
        rplc="N"
    if(postag=="PRP" or postag=="WP"):
        rplc="O"
    if(postag=="NNP" or postag=="NNPS"):
        rplc="^"
    if(postag[0]=="V" or postag=="MD"):
        rplc="V"
    if(postag[0]=="J"):
        rplc="A"
    if(postag[0]=="R" or postag=="WRB"):
        rplc="R"
    if(postag=="UH" or postag=="UH"):
        rplc="!"
    answer=re.match(r'WP$',postag)
    answer2=re.match(r'PRP$',postag)
    if(answer!=None or answer2!=None or postag=="DT" or postag=="WDT" or postag=="PRP$"):
        rplc="D"
    if(postag=="IN" or postag=="TO"):
        rplc="P"
    if(postag=="CC" or postag=="CC"):
        rplc="&"
    if(postag=="RP"):
        rplc="T"
    if(postag=="EX" or postag=="PDT"):
        rplc="X"
    if(postag=="CD"):
        rplc="$"
    if(postag=="FW" or postag=="POS" or postag=="SYM" or postag=="LS"):
        rplc="G"
    if(postag in string.punctuation or postag=="''" or postag=="``" or postag=="-LRB-" or postag=="-RRB-"):
        rplc=","
    if(postag=="HT"):
        rplc="#"
    if(rplc==""):
        rplc=postag
    return rplc
    
##with open("test.predict",encoding="utf-8") as f:
##    for line in f:
##        c=c+1
##        a=line.replace("\n","")
##        b=a.split("\t")
##        #print (a.split("\t"))
##        form=bytes(str(b[1]).encode("utf-8"))
##        form=form.decode("utf-8")
##        print(form)
##        extract_holder.append(Tree(b[0], 
##form,
##b[2],
##b[3],
##b[4],
##b[5],
##b[6],
##b[7],
##b[8],
##b[9]))

line_holders=[]
sentence_holder=[]
files_to_open=["output3.txt","test.predict"]
c=0
for file in files_to_open:
    with open(file,encoding="utf-8") as f:
        hold=f.read()
        trees=hold.split("\n\n")
        #print("+++"+str(len(trees)))
        for tree in trees:
            line_holders.append(tree.split("\n"))
        #print(len(line_holders))
        for tree in line_holders:
            
            for sentence in tree:
                b=sentence.split("\t")
                #print(b[1])
                rplc=""
                if(file=="test.predict"):
                    sentence_holder.append(parseTree(b[0],b[1],b[2],b[3],b[4],b[5],b[6],b[7]))
                else:
                    #print(sentence)
                    rplc=lookup(b[3])
                    sentence_holder.append(parseTree(b[0],b[1],b[2],rplc,rplc,b[5],b[6],b[7]))

            #print(sentence_holder)
            extract_holder.append(deepcopy(sentence_holder))                           
            sentence_holder.clear()
        line_holders.clear()

    if(c==0):
        list1_to_compare=deepcopy(extract_holder)
        extract_holder.clear()

    if(c==1):
        list2_to_compare=deepcopy(extract_holder)
        extract_holder.clear()
    c=c+1
    #print(c)
    extract_holder.clear()


i_j_holder=["99"]
c=0
while c<2:
    if c==0:
        x=list1_to_compare
    if c==1:
        x=list2_to_compare
    for trees in x:
        #print(trees)
        for id1,word in enumerate(trees):
            for id2, word2 in enumerate(trees):
                #print(word2)
                            
                #print(word2.id)
                if(str(id1)+str(id2) not in i_j_holder):
                    if(word.wordID==word2.parentID):
                        word.add_child(word2)
                        i_j_holder.append(str(id2)+str(id1))
                    
                    if(word2.wordID==word.parentID):
                        word2.add_child(word)
                        i_j_holder.append(str(id2)+str(id1))

        i_j_holder.clear()


    for trees in x:
        for word in trees:
            if(word.parentID=="0"):
                #print("mert")
                root=word

    c+=1
root1=None
root2=None
for i in range(len(list1_to_compare)):
    for j in range(len(list1_to_compare[i])):
        if(list1_to_compare[i][j].parentID=="0"):
            root1=list1_to_compare[i][j]
    for j in range(len(list2_to_compare[i])):
        if(list2_to_compare[i][j].parentID=="0"):
            root2=list2_to_compare[i][j]
    if(root1 and root2):
        root1.printNode(root1)
        root2.printNode(root2)
        dist = zss.distance(root1, root2, parseTree.get_children, insert_cost, remove_cost, update_cost)
        print(str(i)+": Distance is "+str(dist))
        
    root1=None
    root2=None
##
##for trees in extract_holder:
##    for word in trees:
##        if(len(word.upostag)>1):
##            #print(word.upostag)

    #print(RenderTree(root))
    #print("\n*************")
#root.show_child()

11	offset	V	0	ROOT
10	won’t	V	0	_
0: Distance is 54
4	is	V	0	ROOT
12	Read	V	0	_
1: Distance is 67
18	…	N	0	ROOT
2	suggest	V	0	_
2: Distance is 37
2	says	V	0	ROOT
2	says	V	0	_
3: Distance is 13
4	is	V	0	ROOT
7	and	&	0	_
4: Distance is 30
3	abandons	V	0	ROOT
3	abandons	V	0	_
5: Distance is 12
19	…	N	0	ROOT
14	should	V	0	_
6: Distance is 44
18	…	N	0	ROOT
3	amplify	V	0	_
7: Distance is 39
21	…	N	0	ROOT
13	will	V	0	_
8: Distance is 54
3	extends	V	0	ROOT
3	extends	V	0	_
9: Distance is 19
3	comes	V	0	ROOT
8	including	V	0	_
10: Distance is 26
10	must	V	0	ROOT
10	must	V	0	_
11: Distance is 48
3	rated	V	0	ROOT
3	rated	V	0	_
12: Distance is 13
2	hasn’t	V	0	ROOT
17	Or	&	0	_
13: Distance is 54
3	have	V	0	ROOT
3	have	V	0	_
14: Distance is 13
8	has	V	0	ROOT
8	has	V	0	_
15: Distance is 25
1	Future	^	0	ROOT
9	hands	N	0	_
16: Distance is 31
2	is	V	0	ROOT
2	is	V	0	_
17: Distance is 19
3	take	V	0	ROOT
3	take	V	0	_
18: Distance is 27
15	…	N	0	ROOT
2	can	V	0	_
19: Distance is 44
6	is	V	0	ROOT
2	and	&	0	_
20